In [1]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(r'C:\Users\pradh\OneDrive - University of New Mexico\Desktop\Fall 2024\CS521- Data Mining\Data_mining')))

from utils.data_utils import (
    test_train_split,
    split_3_channel,
    short_time_fourier_transform,
    integer_label,
    count_unique_colum_and_vlaues
)

In [2]:
feature_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
label_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

train, test = test_train_split(feature_path, label_path, split=0.1)


In [3]:
train['labels']

9666      P
4188      S
6020     Sn
12929    Pg
15052     P
         ..
14302    Lg
2459     Pg
5436      P
4865     Sn
8761      P
Name: labels, Length: 14672, dtype: object

In [4]:
s_train_features, train_labels = split_3_channel(train)
train_labels = integer_label(train_labels)

s_test_features, test_labels = split_3_channel(test)
test_labels = integer_label(test_labels)

train_features = short_time_fourier_transform(data=s_train_features)
test_features = short_time_fourier_transform(data=s_test_features)

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}
Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [18]:
# Assume the target class you want to augment is class 1
target_class = 4

# Step 1: Isolate the target class
# Find indices where train_labels match the target class
target_indices = np.where(train_labels == target_class)[0]

In [19]:
# Extract the corresponding data for augmentation
data_to_augment = s_train_features[target_indices]
labels_to_augment = train_labels[target_indices]

In [20]:
# Step 2: Define your augmentation functions (add noise, time shift, scaling)
def add_noise(data, noise_level=0.05):
    noise = np.random.randn(*data.shape) * noise_level
    return data + noise

In [21]:
def time_shift(data, shift_max=100):
    shift = np.random.randint(-shift_max, shift_max)
    return np.roll(data, shift, axis=1)  # Shifting along the time axis

In [22]:
def scale_amplitude(data, scale_factor=1.2):
    return data * scale_factor

In [23]:
# Step 3: Apply augmentation to the data for the specific class
augmented_data = []
augmented_labels = []

In [24]:
for sample in data_to_augment:
    augmented_sample = sample.copy()
    
    # Apply different augmentations (you can apply one or multiple augmentations)
    augmented_sample = add_noise(augmented_sample, noise_level=0.05)
    augmented_sample = time_shift(augmented_sample, shift_max=100)
    augmented_sample = scale_amplitude(augmented_sample, scale_factor=1.2)
    
    # Add augmented sample and label to the augmented lists
    augmented_data.append(augmented_sample)
    augmented_labels.append(target_class)  # Same label for augmented data

In [25]:
# Convert augmented data to numpy arrays
augmented_data = np.array(augmented_data)
augmented_labels = np.array(augmented_labels)


In [26]:
# Step 4: Reintegrate augmented data into the original dataset
s_train_features = np.concatenate((s_train_features, augmented_data), axis=0)
train_labels = np.concatenate((train_labels, augmented_labels), axis=0)

In [27]:
# Step 5: Apply Short-Time Fourier Transform to the augmented training features
# train_features = short_time_fourier_transform(data=s_train_features)
# test_features = short_time_fourier_transform(data=s_test_features)

In [28]:
np.save("../data/processed/balance_16k/model_input/train_features.npy", train_features)
np.save("../data/processed/balance_16k/model_input/train_labels.npy", train_labels)

np.save("../data/processed/balance_16k/model_input/test_features.npy", test_features)
np.save("../data/processed/balance_16k/model_input/test_labels.npy", test_labels)

In [29]:
print(f"Train features: {train_features.shape}, Train label: {train_labels.shape} ")
print(f"Test features: {test_features.shape}, Test label: {test_labels.shape} ")

Train features: (15854, 3, 50, 50), Train label: (18218,) 
Test features: (1631, 3, 50, 50), Test label: torch.Size([1631]) 


In [5]:
unique_value, count = count_unique_colum_and_vlaues(train_labels)
for x in range(len(count)):
    print(unique_value[x], count[x])

0 2696
1 2713
2 2711
3 2683
4 1174
5 2695
